In [1]:
# initialize modules and globals
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os
import shutil
driver = webdriver.Chrome()

In [2]:
filetype = 'studentdiscipline'
local_html = './data/' + filetype + '/html/html.htm'
yearID = 'ctl00_ContentPlaceHolder1_ddYear'
offenseID = 'ctl00_ContentPlaceHolder1_ddOffense'
groupID = 'ctl00_ContentPlaceHolder1_ddStudentGroup'
exportName = 'ctl00_ContentPlaceHolder1_spExport'
rawfileName = 'StudentDisciplineDataReport.xlsx'

In [3]:
# load html to data
with open(local_html) as html:
    soup = BeautifulSoup(html, 'html.parser')

# scrape available years, grades, and student groups
years = []

for item in soup.find(id=yearID).children:
    if item.string != '\n':
        years.append(item.string)

print(years)

offenses = []

for item in soup.find(id=offenseID).children:
    if item.string != '\n':
        offenses.append(item.string)
        
print(offenses)        

groups = []

for item in soup.find(id=groupID).children:
    if item.string != '\n':
        groups.append(item.string)

print(groups)

['2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13']
['All Offenses', 'Bullying', 'Destruction of school property due to arson', 'Felony conviction or complaint', 'Homicide (murder or manslaughter)', 'Illegal substances', 'Kidnapping (abduction)', 'Non-drug, non-violent or non-criminal-related offense', 'Other violence or substance-related offense', 'Physical attack (battery)', 'Physical fight ', 'Robbery using force', 'Sexual assault', 'Sexual harassment', 'Theft (school, staff or student property)', 'Threat of physical attack', 'Threat of robbery', 'Vandalism/destruction of property', 'Weapon on school premises']
['All Students', 'English Learner', 'Economically disadvantaged', 'Students w/disabilities', 'High needs', 'Female', 'Male', 'Amer. Ind. or Alaska Nat.', 'Asian', 'Afr. Amer./Black', 'Hispanic/Latino', 'Multi-race, Non-Hisp./Lat.', 'Nat. Haw. or Pacif. Isl.', 'White']


In [ ]:
# loop through all combinations of year/grade/group, downloading each file and renaming it
# recreate html page, with each option selected
for year in years:
    for offense in offenses:
        for group in groups:
            for item in soup.find(id=yearID).children:
                try:
                    if item.string == year:
                        item['selected'] = 'selected'
                    if item.string != year:
                        del item['selected']
                except TypeError:
                    pass
            for item in soup.find(id=offenseID).children:
                try:
                    if item.string == offense:
                        item['selected'] = 'selected'
                    if item.string != offense:
                        del item['selected']
                except TypeError:
                    pass
            for item in soup.find(id=groupID).children:
                try:
                    if item.string == group:
                        item['selected'] = 'selected'
                    if item.string != group:
                        del item['selected']
                except TypeError:
                    pass
            html_file = open('./data/' + filetype + '/html/temp.html','w')
            html_file.write(soup.prettify())
            html_file.close()
            driver.get('file:///'+os.getcwd()+'/' + '/data/' + filetype + '/html/temp.html')

            # simulate export click with selenium
            download_button = driver.find_element_by_id(exportName)
            download_button.click()

            # wait for file to download
            time.sleep(5)

            # rename file
        #     filename = 'mcas_' + grade.replace('.', '').replace('/', '') + '_' + group.replace('.', '').replace('/', '') + '_' + year.replace('.', '').replace('/', '') + '.xlsx'
            filename = 'ma_'+ filetype + '_' + offense.replace('.', '').replace('/', '') + '_' + group.replace('.', '').replace('/', '') + '_' + year.replace('.', '').replace('/', '') + '.xlsx'
            os.rename('/Users/esegr1/Downloads/' + rawfileName,'/Users/esegr1/Downloads/' + filename)
            shutil.move(os.path.join('/Users/esegr1/Downloads/',filename), os.path.join('/Users/esegr1/Documents/repos/mcpsa/data/' + filetype + '/raw', filename))